In [1]:
import pandas as pd
import os
import re
from tqdm import tqdm
import matplotlib.pyplot as plt
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
import gensim
from gensim.utils import simple_preprocess
import gensim.corpora as corpora
import itertools
from collections import Counter

Chargement des textes et nettoyage

In [2]:
file = '/home/noe/Documents/2022-2023/ENC/Mémoire/Dataframes/PDV_grams.xlsx'

In [3]:
df = pd.read_excel(file)

In [4]:
def livre_id(string):
    new_str = re.sub(r'[A-Za-z]','', string)
    return new_str

In [5]:
df['ID'] = ''
for i in range(len(df)):
    df['ID'][i] = str(df['Livre'][i])+'_'+ str(df['Numéro de la charte'][i])
df = df[['Livre','ID','Date','lemma']]
df['ID'] = df['ID'].apply(livre_id)

/tmp/ipykernel_187048/3455542086.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ID'][i] = str(df['Livre'][i])+'_'+ str(df['Numéro de la charte'][i])
/tmp/ipykernel_187048/3455542086.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ID'] = df['ID'].apply(livre_id)


In [6]:
livre_1 = df[df['Livre']=='Livre 1']
livre_1 = livre_1.reset_index()
livre_1['split'] = livre_1['lemma'].str.split()

livre_2 = df[df['Livre']=='Livre 2']
livre_2 = livre_2.reset_index()
livre_2['split'] = livre_2['lemma'].str.split()

livre_3 = df[df['Livre']=='Livre 3']
livre_3 = livre_3.reset_index()
livre_3['split'] = livre_3['lemma'].str.split()

livre_4 = df[df['Livre']=='Livre 4']
livre_4 = livre_4.reset_index()
livre_4['split'] = livre_4['lemma'].str.split()

livre_5 = df[df['Livre']=='Livre 5']
livre_5 = livre_5.reset_index()
livre_5['split'] = livre_5['lemma'].str.split()

livre_6 = df[df['Livre']=='Livre 6']
livre_6 = livre_6.reset_index()
livre_6['split'] = livre_6['lemma'].str.split()

In [213]:
"""def remove_str(liste):
    strings_to_remove = ['sum1','habeo','possum1','regnum','Ecclesia','fides2','imperium','Romanus','dico2',
                         'princeps1','facio','dominus','debeo']
    new_list = [x for x in liste if x not in strings_to_remove]    
    return new_list
"""

In [214]:
"""livre_1['split'] = livre_1['split'].apply(remove_str)"""

Topic modeling gensim

In [7]:
dictionary = Dictionary(livre_1['split'])
dictionary.filter_extremes(no_below=5, no_above=0.5)
corpus = [dictionary.doc2bow(tokens) for tokens in livre_1['split']]
tfidf = TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
from gensim.models.ldamodel import LdaModel
lda_model = LdaModel(corpus=corpus_tfidf, id2word=dictionary, num_topics=5, passes=20, alpha='auto')

top = []

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))
    top.append(topic)

Topic: 0 
Words: 0.007*"cura" + 0.007*"permitto" + 0.006*"loquor" + 0.006*"exigo" + 0.006*"committo" + 0.006*"dignitas" + 0.006*"uotum" + 0.006*"humilitas" + 0.005*"gladius" + 0.005*"auspicium"
Topic: 1 
Words: 0.006*"prefero" + 0.005*"teneo" + 0.005*"excogito" + 0.005*"aduersarius1" + 0.004*"littera" + 0.004*"excellentia" + 0.004*"apostolicus" + 0.004*"precedo" + 0.004*"oculus" + 0.004*"turbo2"
Topic: 2 
Words: 0.009*"duo" + 0.009*"apostolicus" + 0.007*"processus" + 0.006*"discrimen" + 0.006*"refero" + 0.006*"magnus" + 0.006*"admitto" + 0.006*"destino" + 0.006*"officium" + 0.005*"reputo"
Topic: 3 
Words: 0.005*"debitum" + 0.005*"ecclesia" + 0.004*"catholicus" + 0.004*"clericus" + 0.004*"scandalum" + 0.004*"officium" + 0.004*"hereticus" + 0.004*"consideratio" + 0.004*"iurisdictio" + 0.004*"prefero"
Topic: 4 
Words: 0.006*"dico2" + 0.005*"rex" + 0.005*"pater" + 0.005*"tempus1" + 0.005*"ciuitas" + 0.005*"honor" + 0.005*"Christianus" + 0.004*"uenio" + 0.004*"ius1" + 0.004*"iustitia"


In [8]:
from gensim.models import CoherenceModel

# Compute coherence score
coherence_model_lda = CoherenceModel(model=lda_model, texts=livre_1['split'], dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.3314665938237292


In [9]:
my_list = []

for i in range(len(top)):
    my_list.append(top[i].split('+'))
my_list = [item for sublist in my_list for item in sublist]
result = []
weight = []
for string in my_list:
    match = re.search(r'"([^"]+)"', string)
    if match:
        result.append(match.group(1))

weight.append([x.split('*')[0] for x in my_list])


In [10]:
weight = list(itertools.chain(*weight))
weight = [string.strip() for string in weight]

In [11]:
def yolo(words, dictionary):
    keys = []
    for word in words:
        key = find_key(word, dictionary)
        if key is not None:
            keys.append(key)
        else:
            new_key = input(f"Enter a key for '{word}': ")
            if new_key not in dictionary:
                dictionary[new_key] = []
            dictionary[new_key].append(word)
            keys.append(new_key)
    return keys


Liste des topics par Sébastien Barret

In [12]:
def topic_list(text):
    text = text.split(';')
    for i in range(len(text)):
        index = text[i].index('(')
        text[i] = text[i][:index]
        text[i] = text[i].replace(' ','')
    for i in range(len(text)):
        if ',' in text[i]:
            split_values = text[i].split(',')
            text.pop(i)
            for j in range(len(split_values)):
                text.insert(i+j, split_values[j])
    text = sorted(list(set(text)))
    return text

In [13]:
abondance = 'abundans (abondant) ; abundanter (abondamment) ; abundantia (abondance) ; abundantius (très abondamment) ; abundo (être plein de, abonder) ; accresco (accroître) ; adaugeo (augmenter) ; affluencia (affluence) ; affluenter (avec affluence, abondance) ; ager (champ) ; alimentum (aliment, ressources matérielles) ; amplio (augmenter) ; amplior (plus ample, plus large) ; amplius (amplement, largement) ; augeo (augmenter) ; augmentatio (augmentation, accroissement) ; augmento (augmenter, accroître) ; augmentum (augmentation) ; copia (abondance) ; copiosior, copiosius (plus abondamment) ; copiosius (très abondemment) ; cresco (croître) ; diminuo (diminuer) ; diminutus (diminué) ; dives (riche) ; egestas (pauvreté, indigence) ; fertilitas (fertilité) ; fructuosus (fructueux, porteur de fruit) ; fructus (fruit) ; frumentum (froment) ; germen (semence) ; germino (germer, croître) ; granatus (bien pourvu en grains) ; horreum (grenier) ; hortus (jardin, verger) ; incrementum (accroissement) ; inedia (privation, manque de nourriture) ; liberalis (généreux, libéral) ; liberalitas (générosité, libéralité) ; liberaliter (généreusement, libéralement) ; liberalius (très généreusement, très libéralement) ; locuples (enrichi, riche) ; locupleto (enrichir) ; magnifice (magnifiquement) ; magnificentia (magnificence) ; magnifico (exalter, surhausser, magnifier) ; magnificus (exaltant, magnifique) ; magnitudo (grandeur) ; major (plus grand) ; meto (récolter) ; munifice (libéralement, généreusement) ; munificentia (libéralité, munificence, générosité) ; munificentior (très libéralement) ; munificus (libéral, généreux) ; oleum (huile) ; pauper (pauvre) ; paupertas (pauvreté) ; pinguior (rendu plus gras, plus abondant, plus riche) ; prodigalitas (prodigalité, dissipation des biens) ; profectus (profit) ; prospere (prospèrement) ; prosperitas (prospérité) ; prosperius (très prospèrement) ; prospero (prospérer) ; prosperus (prospère) ; ros (rosée) ; semino (semer) ; sustento (soutenir, sustenter) thesaurus (trésor) ; uber (abondant, fertile) ; uberior (très abondant) ; uberius (très abondamment) ; ubertas (abondance, fertilité) ; victualia (vivres, denrées comestibles) ; victus (nourriture, subsistance) ; vinum (vin).'
joie = 'beatissimus (bienheureux (pour un saint)) ; beatus (id.) ; benigne (bénignement) ; benignitas (bénignité) ; benigniter (bénignement) ; benivolentia (bienveillance) ; benivolus (bienveillant) ; bravium (récompense, prix) ; consolo (consoler) ; consolatrix (consolatrice (Vierge c.)) ; contentus (content, satisfait) ; cordialis (cordial, venant du cœur) ; delector (se complaire à, avoir plaisir à) ; dilectio (plaisir, délectation, joie à faire) ; dilector (amant (de la majesté royale)) ; dilectus (aimé, favorisé, chéri) ; diligens (diligent, zélé dans la joie) ; diligenter (diligemment, avec enthousiasme) ; diligentia (diligence, enthousiasme) ; diligentius (très diligemment, avec beaucoup d’enthousiasme) ; diligo (aimer, chérir) ; dulcedo (douceur) ; dulcor (douceur) ; felicitas (félicité, joie vive) ; feliciter (avec félicité) ; felicius (avec une grande félicité) ; felix (très heureux) ; flos (fleur) ; fons (source) ; fructifico (donner des fruits) ; fructuosus (qui donne des fruits) ; fructus (fruit) ; fruor (jouir de) ; gaudeo (se réjouir de) ; gaudium (joie, réjouissance) ; germino (germer, faire croître) ; gloria (gloire) ; glorior (se glorifier de, être nimbé par la gloire de) ; gloriose (glorieusement) ; gloriosissimus (id., avec une gloire suprême) ; gloriosius (très glorieusement) ; gloriosus (glorieux) ; gratitudo (gratitude) ; gratus (dont on sait gré) ; lenimentum (allègement, soulagement) ; lenitas (douceur, légèreté) ; letor (être joyeux, se réjouir de) ; libens (dispos, volontaire) ; libenter (volontiers) ; libentior (très volontaire) ; libentius (très volontiers) ; oblectatio (divertissement, joie mondaine) ; placeo (plaire) ; redoleo (embaumer, dégager une bonne odeur) ; refloreo (refleurir) ; relevamen (soulagement, restauration) ; relevo (soulager, restaurer) ; salus (salut (du genre humain)) ; salutaris (porteur de salut) ; splendor (splendeur) ; voluptas (volupté, plaisir).'
amour = 'accendo (au passif, être enflammé par la dévotion) ; adoro (adorer, vénérer) ; affectio (affection, dévotion) ; affecto (avoir en affection, dévotion) ; affectus (adj. d’affection, de dévotion , subst. affection, dévotion) ; alo (nourrir) ; amabilis (aimable, digne d’être aimé) ; amator (amant, qui aime) ; amenitas (caractère de ce qui est digne d’ête aimé) ; amicabiliter (avec amitié) ; amicus (ami) ; amor (amour, dilection) ; amplector (embrasser) ; anxio (être angoissé (pour les sujets)) ; auxilium (secours) ; benivolencia (bienveillance) ; benivolus (bienveillant) ; bonitas (bonté) ; carior (particulièrement cher) ; caritas (charité, participation) ; caritative (avec charité) ; compatior (compatir, souffrir avec) ; condoleo (pleurer avec, partager la souffrance de) ; conforto (consoler, ranimer) ; confoveo (ranimer) ; consolo (consoler) ; cor (cœur) ; cordialis (plein de cœur, cordial) ; cura (sollicitude, soin) ; devote (dévotement, avec dévouement) ; devotio (dévotion, affection, vénération) ; devotius (très dévotement) ; devotus (dévot, fidèle, zélé) ; dilector (ami, qui chéri) ; dilectus (aimé, qui est chéri) ; diligens (diligent, ardent, aimant, zélé) ; diligenter (diligemment) ; diligentia (diligence, zèle) ; diligentius (très diligemment) ; diligo (aimer, chérir) ; fervens (fervent) ; ferventer (de manière fervente) ; ferventius (de manière très fervente) ; ferveo (être fervent dans l’action de) ; fervidus (fervent, zélé) ; fervor (ferveur) ; fidelis (fidèle) ; fidelitas (fidélité) ; fideliter (fidèlement) ; fides (foi) ; fiducia (confiance) ; filialis (filial (dévotion f.le)) ; fraternitas (fraternité) ; inclinatio (inclination) ; libenter (volontiers) ; libentior (qui fait très volontiers) ; libentius (très volontiers) ; paternalis (paternel (amour p.)) ; prefero (préférer, faire passer avant d’autres) ; promptus (prompt (p.te affection)) ; reverencia (révérence, respect, adoration emplis de crainte) ; reventer (avec révérence) ; revereor (révérer) ; sanguis (sang (liens du s.)) ; sinceritas (sincérité (de la dilection, de la foi)) ; sincerus (sincère (affection, dévotion s.)) ; socius (compagnon) ; sollicite (avec sollicitude) ; sollicito (emplir de sollicitude, pousser à une action) ; sollicitudo (sollicitude) ; sollicitus (plein de sollicitude) ; specialis (spécial, particulier (affection, dévotion s.)) ; specialitas (particularité (d’un sentiment)) ; specialiter (spécialement) ; specialius (très spécialement) ; subvenio (venir en aide) ; subventio (aide pécunière) ; succendo (au passif, brûler (d’affection, de dévotion)) ; venerandus (vénérable) ; veneratio (vénération) ; venero (vénérer) ; veneror (vénérer) ; vinculum (lien (de fidélité, du sang, du mariage, etc.)) ; viscerosus (des entrailles, qui touche aux entrailles) ; zelator (zélateur, chérisseur) ; zelo (chérir, être zélé pour) ; zelum (zèle, ardeur pour). '
verticalité = 'altior (plus haut, fort haut) ; altissimus (le Très-Haut) ; altitudo (hauteur, hautesse, altesse) ; altius (fort haut) ; altum (hauteur) ; apex (sommet, faîte) ; aquila (aigle) ; assurgo (se dresser pour une action) ; attollo (exalter) ; caput (tête, chef) ; celestia (les choses d’en haut, célestes) ; celestis/celestus (céleste) ; celitus (du haut des cieux) ; celsitudo (hautesse, altesse) ; celum (ciel) ; conscendo (monter sur) ; culmen (comble, faîte, sommet) ; cumulus (accumulation, comble) ; elevo (élever) ; exaltatio (exaltation, surhaussement, élévation) ; exalto (exalter, surhausser, élever) ; excelsa (les Cieux) ; excelsus (élevé au plus haut des Cieux, très-haut) ; extollo (surhausser, élever, exalter) ; fastigium (comble, faîte) ; gradus (degré) ; grandior (plus grand, plus haut) ; inclino (s’incliner vers) ; insurgo (se dresser pour un acte) ; precelsus (surélevé, suréminent) ; proveho (jucher, exalter, surhausser, transporter vers le haut) ; sol (le soleil dont descend la grâce) ; solium (le trône du haut duquel le roi domine) ; stella (l’étoile du matin, symbole de la Vierge de miséricorde) ; superior (qui est plus haut) ; superius (supérieur, céleste) ; supernus (supérieur, céleste) ; supero (dépasser) ; sursum (vers le haut). '
corps = 'auris (oreille) ; brachium (bras) ; caput (tête) ; caro (chair) ; cor (cœur) ; corpus (corps) ; dextera (dextre) ; incorporalis (incorporel) ; labium (lèvre) ; lingua (langue) ; manus (main) ; membrum (membre) ; oculate (de ses propres yeux) ; oculus (œil) ; os (os) ; palmus (paume de la main) ; pectus (poitrine) ; precordia (diaphragme, cœur, organes centraux de la poitrine) ; sanguis (sang) ; sexus (le sexe, pour la gent féminine) ; sinus (le sein) ; sudor (sueur, transpiration, effort) ; uterus (ventre, utérus) ; viscerosus (qui touche aux entrailles, qui remue les entrailles) ; visio (la vision) ; vulnus (blessure). '
Dieu = 'adjutrix (adjutrice, comme qualificatif de la Vierge) ; almus (bénin, bénéfique, qualificatif de saints) ; altissimus (Très-Haut) ; amator (Dieu, amant et auteur de la paix) ; apostolus (apôtre) ; aquila (aigle, symbole de Dieu) ; auctor (auteur, fondateur) ; beatissimus (bienheureux, qualificatif de saint) ; beatus (bienheureux, qualificatif de saint) ; carno (chair (Dieu fait c.)) celestia (les Cieux) ; celum (ciel) ; Christus (Christ) ; confessor (confesseur (de la foi), qualificatif des saints) ; creator (créateur, qualificatif de Dieu) ; curia (cour céleste) ; Deus (dieu) ; Dominus (le Seigneur) ; dux (Christ comme chef, guide, duc) ; Dyonisius (saint Denis) ; Eleutherius (saint Éleuthère) ; Eva (Éve première parente) ; evangelista (évangéliste) ; excelsus (très-haut, qualificatif de Dieu) ; exercitus (armée, (Dieu des armées)) ; filius (Dieu, fils de Dieu) ; fons (Jésus, source de salvation) ; gloriosissimus (très glorieux, qualificatif de la Vierge et des saints) ; gloriosus (glorieux, qualificatif de la Vierge, des Évangélistes, des saints) ; immaculatus (immaculé, qualificatif de la Vierge) ; Johannes (Jean, évangéliste) ; laurea (laurier de la bénédiction divine) ; Luca (Luc, évangéliste) ; Ludovicus (saint Louis) ; majestas (majesté divine) ; Marchus (Marc, évangéliste) ; margarita (perle incorporelle des dogmes de l’Église) ; Maria (Marie, mère de Dieu) ; martyr (martyr) ; mater (mère Église, Marie, mère de miséricorde) ; Matheus (Matthieu, évangéliste) ; matutinus (matutinal, qualificatif de l’étoile du matin, symbole de Marie) ; omnipotens (Dieu omnipotent) ; pacificus (Dieu, prince faiseur de paix) ; paradisus (paradis) ; pater (Dieu le père) ; patior (souffrir la passion (pour Dieu)) ; patron (saint patron) ; princeps (Dieu, prince faiseur de paix) ; redemptor (Dieu rédempteur de l’humanité) ; regina (la Vierge, reine des cieux) ; rex (Dieu, roi des rois) ; Rusticus (saint Rustique) ; Sabaoth (Dieu des armées) ; Salvator (Dieu sauveur) ; sanctissimus (très saint) ; sanctus (saint, Saint) ; sedes (demeure céleste) ; sidereus (d’étoile, stellaire, qualificatif des demeures célestes) ; spiritus (le Saint-Esprit) ; sponsa (l’Église, épouse du Christ) ; sponsus (Christ, époux de l’Église) ; stella (l’étoile, symbole de Marie) ; supernus (céleste, très-haut) ; Ysaac (Ysaac) ; Yvo (saint Yves).'
jugement = 'accusator (accusateur) ; accusatus (accusé) ; aculeus (trait, aiguillon, pointe (des lois)) ; alienatio (aliénation de patrimoine) ; alienatus (aliéné) ; arrestum (arrêt du parlement) ; canon (canon ecclésiastique) ; clemens (clément) ; clementer (avec clémence) ; clementia (clémence) ; consuetudo (coutume, usage) ; corroboro (renforcer (justice et chevalerie se renforcent mutuellement)) ; delator (délateur) ; delictum (délit) ; dissuetudo (désuétude (des canons)) ; eque (équitablement, justement) ; equitas (équité, justice) ; excommunicatio (excommunication) ; executio (exécution, application de la justice) ; exerceo (exercer la justice) ; forefactum (forfait) ; franchisia (franchise) ; gratia (grâce) ; humiliter (humblement (implorer la grâce h.)) ; illegitime (illégitimement) ; judex (juge) ; judicialis (judiciaire) ; judicium (jugement) ; judico (juger) ; juridictio (juridiction) ; jus (droit) ; juste (avec justice) ; justiciabilis (justiciable) ; justifico (juger innocent) ; justissimus (très juste) ; justus (juste, équitable) ; legalis (légal) ; legatio (légation) ; legitimacio (légitimation) ; legitimus (légitime) ; lesio (entorse (au droit)) ; lex (loi) ; liberalis (généreux (dans l’exécution de la grâce)) ; liberalitas (libéralité, générosité (dans l’exercice de la grâce)) ; liberaliter (avec libéralité, générosité) ; liberalius (très libéralement, très généreusement) ; libertas (liberté, au sens de franchise) ; lis (querelle, procès) ; merus (pur, plein (effet de pure bonté)) ; ministro (administrer la justice) ; misericordia (miséricorde) ; misericorditer (avec miséricorde) ; misericors (miséricordieux) ; mitigo (tempérer (la rigueur de la justice)) ; nego (nier (la grâce aux suppliants)) ; norma (la norme, la loi) ; pena (peine, châtiment) ; prejudicium (préjudice (fait à la juridiction royale)) ; privilegium (privilège) ; querelose (avec caractère de plainte) ; recte (justement, correctement) ; rectitudo (rectitude, droit sens de justice) ; rectus (droit) ; rigor (rigueur (des lois, de la justice)) ; supplex (suppliant) ; supplicatio (supplique) ; suppliciter (en suppliant (implorer en s.)) ; supplico (supplier) ; tempero (tempérer (la rigueur de justice)) ; tribunal (tribunal) ; vindictam (vengeance). '
faute = 'anfractus (les crevasses (des querelles / procès)) ; audacia (audace (de faire des procès)) ; calliditas (ruse, fourbe) ; casus (cas, accident) ; cautela (pris en mal : fraude) ; collabor (chuter, dévier, être entraîné vers le mal) ; collapsus (déchu, qui a chuté, qui a fauté, qui a été entraîné au mal) ; conatu (un entraînement, une impulsion) ; continuo (continuellement (être entraîné à pêcher c.)) ; contrittus (contrit (délinquant c.)) ; convicium (scandale, insulte, altercation, etc.) ; cornu (la corne des superbes, rompue par la justice divine) ; crassans (tyran, malfaiteur) ; crimen (crime) ; criminosus (criminel) ; culpa (faute) ; damnificatus (damné) ; damno (damner) ; damnosus (damnable) ; damnum (le dégât, le dam) ; degenero (dégénérer, s’abâtardir) ; delictum (le délit, la faute) ; delinimentum (séduction (qui conduit à la faute)) ; delinquens (délinquant, fautif) ; delinquo (fauter, aller contre le droit) ; demeritum (démérite, incapacité à gagner des mérites auprès de la divinité) ; devio (dévier, s’écarter du droit chemin) ; discordo (dévier du droit chemin) ; diverto (id.) ; dolosus (frauduleux, trompeur) ; duricia (dureté des pêcheurs) ; elido (rompre (les cornes des orgueilleux)) ; facinus (crime, délit) ; falsus (faux, hypocrite) ; ferreus (de fer (dureté des pêcheurs)) ; fetidus (puant, qualifiant les boucs et brebis galeux) ; fomes (aliment du feu (de la cupidité)) ; fornicor (forniquer, s’adonner à la prostitution) ; fragilitas (fragilité (de la nature humaine, prompte au mal)) ; fraus (fraude) ; gravamen (oppression) ; hedo (bouc, symbole d’impureté) ; ignorantia (l’ignorance qui séduit) ; impietas (impiété) ; improbus (malhonnête, de mauvaise conduite) ; indebite (indûment) ; indebitus (indu, qui ne devrait pas être fait) ; iniquitas (iniquité) ; iniuria (injure, insulte, tort infligé contre tout droit) ; iniuriosus (injurieux, infligeant un tort contre tout droit) ; inquinamentum (pollution, souillure) ; insidia (ruse, traquenard) ; insidiosus (insidieux) ; insolentia (insulte) ; iracundia (colère) ; labes (glissade vers la faute, faute, chute dans la faute, écart) ; labilis (glissant (condition humaine glissant facilement vers le mal)) ; lapsus (chute (d’Adam)) ; limes (limite que le fautif dépasse) ; lubricus (glissant (vers la faute)) ; macula (tache d’opprobre) ; maculo (entacher d’opprobre) ; malefactum (le méfait) ; malicia (la méchanceté) ; malignans (le méchant) ; malignus (méchant, pervers) ; malum (le mal) ; malus (le mauvais, le méchant) ; mortiferus (porteur de mort, mortifère (le fruit de l’arbre de science)) ; nephandus (néfaste, maudit) ; nocens (coupable) ; nubes (nuage (de l’ignorance)) ; obtenebro (enténébrer : la faute d’Adam enténèbre l’esprit humain) ; odium (haine) ; offensa (offense) ; ovis (la brebis, quand elle est galeuse) ; patrator (auteur (d’un crime)) ; peccans (pécheur) ; peccator (pécheur) ; peccatum (péché) ; pecco (pêcher) ; penitens (pénitent) ; penitentia (pénitence) ; peniteo (se repentir) ; perfidus (perfide) ; perpetro (perpétrer, commettre (un crime)) ; perversi (les pervers, les méchants) ; perversitas (perversité, méchanceté, malice) ; pronus (prompt, enclin (au mal)) ; reus (coupable) ; sarcina (le fardeau (des péchés)) ; sceleratus (scélérat) ; scelus (crime) ; superbus (orgueilleux) ; temptacio (tentation) ; transgressor (transgresseur, outrepasseur de ses droits). '
gouvernement = 'assisto (assister (le roi)) ; committo (confier (le gouvernement) à) ; conamen (effort) ; confirmo (confirmer (un acte)) ; corrigo (corriger) ; decerno (décider, décréter) ; defendo (défendre) ; defensio (défense) ; defenso (défendre) ; deliberatio (délibération) ; diligentius (avec diligence) ; dispono (disposer, ordonner) ; dispositio (disposition, ordonnancement) ; domanium (domaine (de la couronne)) ; dominium (domaine où s’exerce l’autorité, autorité) ; dominor (dominer, régner) ; dominus (seigneur) ; domus (maison (parfois comme métaphore du royaume)) ; edictum (édit) ; executio (exécution (de la justice)) ; exequor (exécuter (la justice)) ; exerceo (exercer (le devoir, le gouvernement, la coutume)) ; exercitium (exercice) ; facere (faire) ; factum (fait, haut-fait, conduite, action) ; feliciter (heureusement (régner h.)) ; felix (heureux (gouvernement h.)) ; fundo (fonder) ; fungor (s’acquiter de, exercer) ; gesta (les gestes, hauts-faits (des rois)) ; gratia (grâce, cf. section suivante) ; gubernaculum (gouvernail, et par métaphore, gouvernement) ; guberno (gouverner) ; habena (les rênes (du royaume de France)) ; impa/ertio(r) (délivrer, donner, impartir) ; impendo (consacrer à) ; imperium (empire, domination) ; impero (commandement) ; impono (imposer) ; incumbo (incomber, revenir (au roi de faire telle action)) ; insudo (peiner, faire violemment et continuement effort pour) ; labor (travail, labeur) ; laboro (travailler) ; ministerium (ministère, office, office sacral (de la royauté)) ; moderamen (gouvernement, maîtrise) ; moderor (gouverner, maîtriser) ; nitor (s’efforcer de) ; onus (poids, charge (du gouvernement royal)) ; opus (œuvre, travail, occupation (royal/e)) ; ordino (ordonner, disposer) ; pervigilis (très vigilant, toujours vigilant) ; petitio (demande, requête (faite au roi, cf. 11. La grâce)) ; prebeo (accorder (le consentement royal)) ; prefero (préférer) ; procuro (procurer) ; propono (se proposer de, se disposer à) ; prosequor (poursuivre, favoriser) ; protectio (protection) ; protego (protéger) ; provideo (pourvoir à, prévoir) ; providus (prévoyant, prudent, clairvoyant) ; provisio (prévoyance, prudence, provision) ; prudens (prudent) ; prudenter (prudemment, sagement) ; prudentia (prudence, sagesse) ; publicus (public) ; regimen (gouvernement) ; regnans (r. régnant) ; regno (régner) ; regnum (royaume) ; rego (régir, régner, gouverner) ; relevo (soulager, restaurer (par son aide royale)) ; removeo (ôter, enlever, supprimer) ; respublica (république, équivalent lettré de ‘royaume’) ; sollicitudo (soin, vigilance) ; subvenio (subvenir, secourir) ; sustineo (soutenir, endurer) ; tueor (conserver, garder, préserver, respecter) ; tutela (tutelle, garde) ; tutius (très sûrement) ; utilitas (utilité, bien commun) ; utiliter (utilement) ; vaco (travailler à) ; vigilius (constant, ardent, régulier, qui s’exerce jour et nuit).'
grâce = 'attollere (exalter) ; beneficium (bénéfice) ; benemerens (méritant, ayant bien mérité) ; benemereor (bien mériter) ; benemeritum (action méritant récompense) ; benemeritus (sujet méritant, ayant bien mérité) ; benignitas (bonté (royale)) ; benigniter (avec bonté) ; benignus (gracieux, bénin, bon) ; benivolentia (bienveillance) ; benivolus (bienveillant) ; bravium (prix de la victoire, récompense) ; clemens (clément) ; clementer (avec clémence) ; clementia (clémence) ; concedo (concéder) ; concessio (concession) ; condignus (digne, convenable (grâce, récompense convenable)) ; condimentum (assaisonnement (de la grâce, tempérant la rigueur)) ; confirmo (confirmer (un acte)) ; copiosior (très abondant, large) ; decerno (décréter) ; dextera (dextre (de la grâce royale)) ; donum (don) ; dos (dot) ; doto (doter) ; elargior (donner libéralement) ; elargitio (don) ; elargitus (qui a été donné par grâce) ; exaltatio (exaltation (du statut)) ; exaudio (exaucer) ; exauditio (exaucement) ; exhibeo (se montrer (enclin à la grâce)) ; exhibitio (démonstration (de grâce)) ; extendo (étendre (la main de la clémence, de la faveur royale)) ; extollo (exalter, exhausser) ; favor (faveur) ; favorabilis (favorable) ; favorabiliter (favorablement) ; favoriter (id.) ; fructuosus (fructueux (faveur fr.s)) ; gratanter (volontiers, grâcieusement) ; grate (avec reconnaissance) ; gratia (grâce) ; gratiose (gracieusement) ; gratiosius (très gracieusement) ; gratiosus (gracieux) ; gratis (gracieusement, à titre gracieux) ; gratissimus (très reconnaissant, susceptible d’une grande reconnaissance) ; gratitudo (reconnaissance, gratitude) ; gratuitus (gracieux, librement accordé) ; gratus (digne de grâce, gracieux, digne de reconnaissance) ; honorificentia (grâce accroissant l’honneur) ; honoro (honorer) ; impa/ertio(r) (délivrer (la grâce)) ; impendo (rendre (grâce)) ; inclino (incliner (les oreilles royales vers les demandes de grâce)) ; insignio (distinguer) ; intitulo (revêtir d’un titre) ; largior (donner, répandre généreusement) ; largissimus (très généreux) ; largitas (largesse) ; largitio (action de délivrer largement) ; largitor (celui qui donne libéralement, largement) ; largus (libéral, large) ; liberalior (très libéral, très généreux) ; liberalitas (générosité (royale)) ; liberaliter (libéralement, généreusement) ; liberalius (très libéralement) ; magnificatus (magnifique) ; magnifice (magnifiquement, avec magnificence) ; magnificentia (magnificence) ; magnifico (exalter) ; magnificus (magnifique) ; mater ((Marie), mère (de grâce)) ; mereo (mériter : cf. 15. Le mérite) ; misericordia (miséricorde) ; misericorditer (miséricordieusement) ; misericors (miséricordieux) ; mitigo (tempérer, mitiger la rigueur (par la grâce)) ; munificentia (munificence (de la grâce royale)) ; munus (présent, don) ; ostendo ((se) montrer (gracieux)) ; parco (épargner) ; penitens (pénitent) ; penitentia (pénitence) ; petitio (requête, requête gracieuse) ; peto (demander) ; pietas (pitié) ; porrigo (tendre en avant (les mains de la largesse, libéralité, etc.)) ; prebeo (accorder, procurer (la grâce)) ; premio (récompenser) ; premium (récompense) ; prerogativa (prérogative) ; prevenio (prévenir (par sa grâce)) ; privilegium (privilège) ; prosequor (poursuivre (de ses grâces)) ; recompensatio (récompense, compensation extraordinaire) ; reficio (restaurer, soulager) ; relevamen (soulagement, restauration) ; relevo (relever, restaurer (par sa grâce)) ; remuneracio/renumeratio (récompense, rémunération) ; renumerator (sic pour remunerator, rémunérateur) ; renumeratus (sic pour remuneratus, rémunéré, récompensé) ; retributio (rétribution) ; rigor (rigueur de justice, tempérée par la grâce) ; specialis (spécial) ; specialitas (caractère spécial (qui motive une grâce spéciale, un privilège)) ; specialiter (spécialement) ; specialius (très spécialement) ; supplex (suppliant) ; supplicatio (supplication) ; suppliciter (en suppliant) ; supplico (supplier) ; tempero (tempérer (la rigueur par la grâce)) ; titulus (titre (d’honneur, de noblesse)) ; unguentum (le baume, l’onguent (de la grâce)). '
guerre = 'accusator (accusateur) ; adverso (combattre en ennemi) ; adversitas (adversité, malheur) ; adversum (malheur, fortune adverse) ; adversus (contre) ; afflictio (affliction, malheur causé par la guerre) ; arma (armes) ; bellator (guerrier) ; bellicus (belliqueux, guerrier) ; bellum (guerre) ; brachium (bras (qui frappe)) ; calamitas (malheur, désastre de la guerre) ; calliditas (ruse, malice) ; capio (capturer) ; casualis (fortuit, pour une défaite) ; clades (combat) ; clipeus (bouclier) ; contentio (lutte) ; cornu (corne des orgueilleux, superbes, insolents dans le combat) ; damnum (dommage) ; defendo (défendre) ; defensio (défense du royaume) ; defenso (défendre) ; defensor (défenseur) ; depredo (piller) ; destructio (destruction) ; devinco (vaincre) ; dimico (combattre, lutter) ; discrimen (mauvaises passes, instants critiques) ; dissensio (dissension, discorde) ; emulus (envieux, ennemi par envie) ; eventus (fait d’armes) ; exercitus (armée) ; felix (heureux = épisode guerrier h.) ; fortitudo (courage, force) ; gloria (gloire, dont gloire de la chevalerie) ; guerra (guerre) ; incursus (incursion, pillage) ; inimicus (ennemi) ; insultus (insultes, assauts des ennemis) ; insurgo (se lever en armes) ; invasio (invasion) ; invasor (envahisseur) ; jurgium (dispute, rixe) ; litigo (se quereller, se faire des procès) ; malignus (méchant, caractérisant les actes des ennemis) ; miles (chevalier) ; militia (chevalerie) ; propulso (expulser, repousser) ; rabies (rage (de la discorde)) ; rebellis (rebelle) ; seditio (la sédition, la rébellion) ; sudor (transpiration, dans le sens d’efforts guerriers) ; superbus (orgueilleux (ennemi)) ; tempestas (tempête de la sédition, de la rébellion) ; turbatio (bouleversements de la guerre) ; victor (vainqueur) ; victoria (victoire) ; victoriosus (victorieux) ; viriliter (mâlement) ; zizania (discorde, zizanie). '
imitatio = 'consuesco (être habitué à, avoir coutume de) ; consuetudo (coutume) ; degenero (dégénérer, ne pas suivre l’exemple de sa lignée) ; exemplar (exemple, exemplaire, image à imiter) ; exemplum (exemple, modèle) ; gesta (gestes, hauts faits (des prédécesseurs, à imiter)) ; imago, ymago (image) ; imitatio (imitation) ; imitator (imitateur) ; imitatrix (imitatrice) ; imitor (imiter) ; immitto, is (id.) ; inhereo (suivre, entrer dans les pas de) ; insecto (poursuivre, suivre) ; insequor (poursuivre, suivre) ; predecessor (prédécesseurs, à imiter) ; reformer (redonner sa forme première) ; retroactus (passé (temps p. à imiter)) ; semita (trace, chemin) ; sequor (suivre, poursuivre) ; soleo (faire traditionnellement, avoir coutume de faire) ; solitus (habituel, normal) ; speculum (miroir) ; tempus (temps (passé, à imiter)) ; vestigium (traces, voie (à suivre)).'
chaleur = 'accendo (allumer) ; ardentius (très ardemment) ; ardeo (brûler) ; ardor, ris (ardeur, flamme) ; caligo (obscurité) ; calor (chaleur) ; candelabrum (candélabre) ; clare (clairement, de manière éclatante) ; claresco (éclater) ; clarior (très clair, éclatant) ; clarissimus (très clair, illustre) ; claritas (clarté, éclat) ; clarus (clair) ; extinguo (éteindre) ; fulgens (éclatant comme l’éclair) ; fulgor (éclair) ; gemma (pierre précieuse, gemme) ; illustris (illustre, éclatant) ; illustro (rendre illustre, couvrir de lumière) ; inextinguibilis (inextinguible) ; lumen (lumière) ; margarita (perle) ; matutinus (matinal, matutinal, associé à l’éclat de l’étoile) ; nebulosus (recouvert par un nuage qui obscurcit) ; nitor (éclat) ; nubes (nuage qui obscurcit) ; obtenebro (enténébrer) ; preclaro (être plus éclatant que les autres) ; preclarus (plus éclatant que les autres, particulièrement éclatant) ; preluceo (éclater, luire particulièrement) ; prerutilo (flamboyer plus fortement, flamboyer plus que les autres) ; refulgeo (éclater) ; sidereus (d’étoile, qui a l’éclat d’une étoile) ; stella (étoile) ; succendo, is (enflammer) ; tenebre (ténèbres).'
noblesse = 'adorno (orner (son âme de vertus)) ; affinitas (lien de sang, lien d’alliance) ; antecello (dépasser les autres (en vertus, services, grandes actions)) ; arma (armes (faits d’a.)) ; baro (les grands nobles, les barons) ; bellator (combattant, guerrier, comme synonime de noble) ; collateralis (compagnon étroit (du roi)) ; comes (comte) ; consanguinitas (lien du sang) ; decor (prééminence morale, illustration par les mœurs et l’action) ; decoro (décorer, orner, embellir (d’un titre, d’un comté, d’une pairie, etc.)) ; degenero (dégénérer, perdre les vertus de son lignage) ; dignitas (dignité, rang, noblesse) ; exorior (naître (du sang royal)) ; exposco (s’exposer (pour le royaume)) ; fama (renommée) ; fedus (alliance) ; genitura (naissance) ; genus (race, lignage) ; inclitus (éclatant, insigne) ; intitulo (décorer de titres, orner de titres) ; milicia (chevalerie) ; nobilis (noble) ; nobilitas (noblesse) ; nobilito (ennoblir) ; nomen (nom, renom) ; persona (personne (noble)) ; preclarus (éclatant, illustre (lignage)) ; preconium (éclat (sonore, de la renommée)) ; probitas (probité, vertu, prud’hommerie) ; progenies (souche) ; recommando (recommander (être r.é par ses services pour être ennobli ou exalté)) ; servicium (service rendu, méritant annoblissement ou exaltation du statut) ; titulus (titre d’honneur, titre de gloire (lié à la noblesse)) ; virtus (vertu).'
paix = 'amator (Dieu, amant de la paix) ; amenitas (caractère aimable (de la paix)) ; amplector (embrasser (la paix)) ; auctor (Dieu, auteur de la paix) ; beatitudo (bonheur, état heureux (du repos, de la paix)) ; clementia (clémence (qui donne la paix)) ; concordabilis (facile à accorder, accordable) ; concordia (concorde, bonne entente) ; confirmatio (confirmation, renforcement) ; confirmo (assurer, confirmer, renforcer) ; consilium (conseil, avis (de paix)) ; protego (protéger) ; defendo (défendre) ; defenso (id.) ; dulcor (douceur (de la clémence qui donne la paix)) ; ecclesia (l’Église, principe organisateur) ; exterminium (anéantissement (des vices)) ; fedus (traité) ; fides (foi, confiance) ; firmitas (protection, ferté, muraille) ; firmiter (solidement, sûrement) ; firmo (solidifier, rendre sûr) ; fraternitas (fraternité) ; fraternus (fraternel) ; gradus (grade, degré (d’une hiérarchie ordonnée)) ; instituo (instituer, fonder) ; letor (jouir (de la paix)) ; limes (limite) ; munimen (protection, rempart) ; ordinatus (hiérarchiquement pensé, ordonné) ; osculum (baiser (de la paix et de la justice)) ; ordo (ordre hiérarchique céleste, et son reflet l’ordre hiérarchique social) ; pacificans (qui donne la paix) ; pacifice (pacifiquement, de manière à porter la paix) ; pax (paix) ; prospere (de manière prospère) ; prosperitas (prosperité (des sujets)) ; prosperius (de manière très prospère) ; prosperus (prospère) ; protectio (protection) ; pulchritudo (beauté (de la paix)) ; quies (repos, calme, tranquillité) ; quiesco (reposer en paix, vaquer en paix) ; quietus (calme, en repos, en paix) ; regularitas (respect des règles, manière d’agir/d’être réglé) ; requies (repos, paix) ; rite (conformément aux rites, à l’ordre juste) ; ritus (rite, juste ordonnancement des conduites et cérémonies) ; roboro (solidifier, consolider) ; robur (dureté/solidité de chêne, protection (de la justice)) ; securitas (sûreté) ; stabilisco (établir, constituer en ordre) ; stabilis (stable, dans son assiette) ; stabilitas (stabilité, équilibre, bon ordre) ; stabilius (particulièrement stable) ; status (statut, état, assiette, équilibre correct (du royaume)) ; terrenus (terrestre (paix terrestre)) ; tranquillitas (tranquillité, paix) ; tranquillus (tranquille, en paix) ; tueor (protéger) ; unanimis (unanime, sans exception) ; unanimiter (unanimement) ; unitas (unité) ; unitus (uni) ; vallo (protéger, fortifier, munir d’une­ protection) ; vigilo (veiller à).'
connaissance = 'acies (regard, souvent regard mental) ; afficio (impressionner, pénétrer de) ; anima (âme) ; animo (animer) ; animus (esprit) ; arbitror (estimer, juger bon de) ; archana (entrailles, secrets du cœur, de la poitrine) ; aspiro (aspirer à) ; assensus (consentement) ; attendo (considérer que) ; attente (avec attention, attentivement) ; attentior (très attentivement) ; attentius (avec grande attention, très attentivement) ; censeo (estimer, juger) ; censura (jugement) ; circumspectus (circonspection, prudence, circonspect, prudent) ; circumspicio (contempler) ; cogitatus (méditation, pensée profonde) ; cogito (penser, méditer) ; cognitor (connaisseur, sondeur) ; cognosco (connaître) ; comperio (découvrir, reconnaître) ; conceptum (force de conception) ; conjecturatus (conjecturé, supposé) ; conscientia (conscience) ; consilio (conseiller) ; consilium (conseil) ; conspectus (regard, considération) ; conspicio (considérer, juger bon) ; consulo (réfléchir à) ; consulte (avec réflexion) ; consultum (avis) ; consultus (bien réfléchi) ; cor (cœur) ; credo (croire, croire bon) ; cupio (désirer) ; cura (soin, souci) ; curiose (avec soin, avec acharnement) ; curiosius (avec grand soin, avec beaucoup d’acharnement) ; decerno (décider, décréter) ; decet (il convient de) ; delector (être plein de joie à l’idée de) ; deliberatio (délibération) ; demonstro (démontrer) ; desiderium (désir) ; desidero (désirer) ; dignor (daigner) ; dignosco, dinosco (savoir) ; diligens (diligent, esprit d.) ; docens (enseignant) ; doceo (enseigner) ; doctrina (doctrine, supériorité morale fondée sur la connaissance) ; doctus (docte, savant) ; dogma (dogme, principes de connaissance) ; edoceo (enseigner) ; error (erreur) ; erudio (enseigner, affiner par l’enseignement) ; eruditio (savoir accumulé, savoir qui affine) ; examen (balance de la pensée) ; excogito (pourpenser, imaginer) ; existimo (estimer, juger bon) ; experientia (expérience accumulée par l’esprit) ; experimentum (expérience donnée par les choses) ; experior (apprendre par expérience) ; explano (expliquer, dévoiler) ; extimo (estimer) ; innotesco (devenir clair, être connu) ; instruo (instruire) ; intellectus (intellect) ; intuitus (regard, force de pénétration) ; invenio (trouver, constater) ; liber (livre (de mémoire)) ; meditatio (méditation) ; memini (être conscient de, se souvenir) ; memor (conscient) ; memoria (mémoire) ; memoriter (en mémoire) ; memoro (se souvenir) ; mens (esprit) ; moneo (instruire, avertir) ; oculate (visuellement) ; oculus (œil (de l’esprit, de la pensée, canal de la pensée)) ; penso (penser) ; perfectio (perfection (de la science)) ; prefero (préférer, porter en avant) ; prepono (donner la priorité à) ; probabiliter (avec probabilité) ; propensius (volontiers) ; propono (se proposer de) ; prosequor (poursuivre (un objectif)) ; provideo (pourvoir à, prévoir) ; puto (penser, estimer) ; quotidianus (quotidien (méditation quotidienne)) ; ratio (raison, calcul) ; rationabilis (raisonnable) ; rationabiliter (avec raison, raisonnablement) ; recenseo (repasser en esprit) ; recognitio (recognition, reconnaissance) ; recognosco (reconnaître) ; recolo (repasser en esprit) ; recordatio (souvenir, rappel, évocation) ; recordor (se souvenir, se rappeler) ; rememoro (se souvenir) ; reminisco (se souvenir) ; reperio (trouver) ; reputo (considérer) ; respicio (regarder, considérer) ; revolvo (rouler dans son esprit) ; sagacius (habilement, intelligemment) ; sagax (sage, intelligent) ; sapiens (sage) ; sapienter (sagement) ; sapientia (sagesse) ; schola (école) ; scholaris (écolier) ; scholasticus (des écoles) ; scientia (science) ; scio (savoir) ; secretum (secret (du cœur, de l’esprit)) ; sollercia (habileté, intelligence) ; sollers (intelligent, habile) ; sollerter (intelligemment, habilement) ; sollicite (avec soin, peine) ; sollicitudo (soin, peine) ; sollicitus (attentif, qui prend de la peine) ; speculator (réflexif (Dieu, comme seul être parfaitement réflexif)) ; speculum (miroir (favorisant la pensée)) ; studeo (étudier, enteprendre, s’efforcer de) ; studiose (avec ardeur, effort, étude) ; studiosus (savant, étudiant) ; studium (étude, effort) ; volo (vouloir) ; voluntas (volonté) ; votum (vœu, désir).'
prière = 'devote (dévotement) ; devotio (dévotion, foi) ; devotius (dévotement) ; devotus (dévot) ; dies (jour : prier jour et nuit) ; erogatio (demande, prière) ; erogare (demander, prier) ; exoro (prier, prier fortement) ; famulatus (la prière) ; famulor (prier) ; ferventius (ardemment) ; fructifico (donner des fruits (pour une vie de prière)) ; fructus (fruit (de la pière)) ; humiliter (humblement) ; intercessio (intercession) ; intercessor (intercesseur) ; invoco (invoquer) ; mediator (médiateur, par sa prière) ; oratio (prière, oraison) ; orator (celui qui prie) ; prex (prière) ; supplex (suppliant) ; supplicatio (supplication, prière) ; votivus (votif, de requête, de prière) ; votum (vœu, désir). '
roi = 'altitudo (hauteur (de l’altesse royale)) ; antecessor (prédécesseur (du r.)) ; apex (sommet (de la dignité royale)) ; assensus (approbation royale) ; auctoritas (autorité, pouvoir fondateur, pouvoir législateur royal) ; beneplacitum (bon plaisir royal) ; benignitas (bienveillance, bénignité, bonté royale) ; celsitudo (altesse, hautesse, hauteur royale) ; clarus (illustre, fameux) ; clementia (clémence royale) ; clipeus (bouclier (de la protection royale)) ; consciencia (conscience royale) ; culmen (comble, hauteur, suréminence royale) ; devotio (dévotion portée à la personne du roi) ; devotus (dévot) ; dextera (dextre royale) ; dignitas (dignité royale) ; dominus (seigneur) ; eternus (Dieu, r. éternel) ; excellentia (excellence royale) ; fastigium (faîte, hauteur, éminence royale) ; filius (fils de r.) ; Franci (Francs, Français (r. des F.)) ; Francie (France (r. de F.)) ; gloria (gloire du r.) ; honor (honneur royal) ; illuster (illustre) ; imperiosus (impérieux, paré d’une autorité impériale) ; inclitus (célèbre, fameux, illustre) ; largitas (largesse royale) ; latus ((au) côté (du roi)) ; liberalitas (libéralité, générosité royale) ; majestas (majesté royale) ; manus (main du r.) ; meditatio (méditation royale) ; memoria ((illustre) mémoire (des r.s prédécesseurs)) ; ministerium (ministère, office (sacral de la royauté)) ; mos (mœurs, coutumes royales) ; munificentia (munificence royale) ; onera (regia onera : fatigues royales : fardeau royal) ; pacificus (Dieu, roi faiseur de paix) ; plenitudo (plénitude (de la puissance royale)) ; orbis (r.s de l’orbe (des terres)) ; possibilitas (possibilité, pouvoir d’action (royale)) ; potencia (puissance royale) ; potestas (puissance royale) ; precellentia (excellence, prééminence, suréminence, autorité éminente du r.) ; predecessor (prédécesseur du r.) ; preeminencia (suréminence, éminence royale) ; princeps (prince, doublet rhétorique biblique de r.) ; protectio (protection royale) ; providentia (providence, prévoyance, prudence royale) ; recordatio (mémoire (r.s de France d’illustre mémoire)) ; regalis (royale) ; regius (royal) ; regno (régner) ; reverentia (crainte révérentielle, respect (du r.)) ; rex (roi, dans l’expression Rex regum, pour Dieu) ; sceptrum (sceptre royal) ; serenitas (sérénité royale) ; solercia (intelligence royale) ; stabilisco (établir, constituer (comme r.)) ; summus (suprême (Dieu, r. suprême)) ; supernus (céleste, d’en haut (Dieu, r. d’en haut)) ; thesaurus (trésor royal) ; vestigium (trace (des r.s du passé)).'
royaume ='advena (étranger, forain) ; alienigena (forain, étranger au royaume, né hors du royaume) ; caput (capitale, cité capitale du r.) ; civitas (cité (du royaume)) ; celestis (r. céleste) ; commodum (utilité, profit du r.) ; conservatio (conservation, défense du r.) ; corona (couronne) ; custodia (garde du r.) ; defendo (défendre (le r.)) ; defensio (défense (du r.)) ; defenso (défendre) ; devotus (fidèle du r.) ; distinctio (distinction des r., comme création planifiée des différents royaumes par Dieu) ; Dyonisius (saint Denis, patron du royaume) ; Eleutherius (saint Éleuthère, saint comparse de saint Denis) ; exonero (exonérer (les patrimoines du r.)) ; fastigium (faîte, hauteur suprème, gouvernement du r.) ; fidelis (fidèle du r.) ; finis (frontière, confins du royaume) ; forensis (forain) ; Franci (r. des Francs, ou Français) ; Francia (r. de France) ; fundamenta (fondations, fondements des r.s) ; gloria (gloire du r.) ; gubernaculum (gouvernail du r.) ; guberno (gouverner le r.) ; habena (rênes du r. (tenues par le roi)) ; honor (honneur du r.) ; incola (habitant (du r.) ; inimicus (ennemi (du r.)) ; lilium (lys, symbole du r.) ; militia (chevalerie (rempart des r.s)) ; moderor (gouverner le r.) ; pars (partie du r.) ; patrimonia (patrimoines du r.)) ; patronus (Saint Denis, patron du r. et du roi) ; pax (paix du r.) ; populosus (populeux, bien peuplé) ; presideo (gouverner le, être à la tête du r.) ; propulso (expulser (les ennemis du r.)) ; reformatio (réformation, retour à l’ordre ancien du r.) ; regimen (gouvernement du r.) ; respublica (république, chose publique, affaire publique, équivalent lettré de « royaume » : renforcement dans l’expression « respublica regni nostri ») ; Rusticus (saint Rustique, saint comparse de saint Denis) ; sceptrum (au pluriel, sceptres des r.s) ; status (statut (prospère du r.)) ; subditus (sujet du r.) ; terrenus (terrestre, r. terrestre, par opposition au r. céleste) ; tutamentum (garde, défense, protection du r.) ; tranquillitas (tranquillité du r.) ; tempus (temps anciens du r.) ; utilitas (utilité, profit du r.)'
serviteur = 'affectio (affection, amour) ; alienigena (forains, étrangers) ; amor (amour (des sujets pour le roi)) ; ardor (ardeur (de la dévotion)) ; christianus (chrétien (habitants du r. chrétiens)) ; clericus (clerc) ; constans (constant, fidèle) ; constantia (constance, fidélité) ; continuatus (continu (service continu)) ; devotius (avec un grand dévouement) ; devotus (dévot, fervent) ; diligens (diligent) ; diligenter (avec diligence) ; diligentia (diligence) ; diligentius (très diligent) ; docens (enseignant) ; ecclesiasticus (ecclésiastique (sujets ecclésiastiques, gens d’Église)) ; fervens (fervent) ; ferventius (avec beaucoup de ferveur) ; fervidus (fervent) ; fervor (ferveur) ; fidelis (fidèle, féal) ; fidelitas (fidélité, foi) ; fideliter (fidèlement, avec foi) ; fides (fidélité, foi) ; Franci (les Francs, Français) ; gens (nation, peuple) ; gratitudo (gratitude, reconnaissance) ; gremium (sein (sein du roi, refuge des sujets)) ; grex (troupeau, dans grex Francorum, métaphore des habitants du royaume) ; humiliter (humblement (pour des sujets implorant la grâce royale)) ; imploro (implorer) ; incola (habitant (du royaume)) ; industria (industrie, zèle, application) ; industriosus (appliqué, zélé) ; laudabiliter (de manière digne de louange (qualifiant l’action de serviteurs)) ; libentius (plus volontiers) ; licitus (licite (pour des requêtes de sujets)) ; magister (maître d’université) ; mercator (marchand) ; meritum (mérite) ; obediens (obéissant) ; obediencia (obédience, obéissance) ; obsequium (service, obéissance active) ; obsequor (servir) ; officiarius (officier) ; officium (office, charge) ; ovis (brebis (du troupeau protégé par le roi)) ; perseverans (persévérant (foi, confiance des sujets)) ; perseverantia (persévérance) ; persisto (persister (dans la dévotion au roi)) ; persona (personne) ; populus (peuple) ; promptus (prompt (à servir)) ; puritas (pureté (des sentiments de dévotion, foi, envers le roi)) ; purus (pur (pureté des sentiments de dévotion des sujets)) ; regnicola (habitant du royaume) ; religiosus (religieux) ; scholaris (écoliers, étudiants fréquentant les studia) ; scholasticus (gens des écoles et studia) ; secularius (séculier) ; servio (servir) ; servitium (service (du roi)) ; servitor (serviteur) ; status (statut (des sujets)) ; studens (étudiant) ; subditus (sujet) ; subjaceo (être sujet, assujetti à) ; subjectio (subjection) ; subjectus (sujet) ; supplicatio (supplication, requête) ; suppliciter (avec supplication) ; supplico (supplier) ; vir (homme (qualifiant des sujets de genre divers, éminents)) ; zelator (fervent soutien, serviteur/sujet zélé) ; zelo (être zélé) ; zelum (zèle).'
musique = 'canticus (cantique) ; consono (être cohérent avec : consonner) ; consonus (cohérent, consonnant) ; dissonus (dissonant, non-cohérent) ; exaudio (écouter et exaucer) ; fama (renommée) ; harmonia (harmonie) ; hymnus (hymne) ; missa (messe) ; praeconium (cri, cri du crieur, annonce sonore) ; propalo (rendre public) ; sonus (son) ; sonorus (sonore).'
biodiversite = 'agnus (agneau) ; animantia (les être vivants en général) ; aqua (eau) ; aquila (aigle) ; cornu (corne) ; equus (cheval) ; fera (bête féroce, fauve) ; flos (fleur) ; fructus (fruits) ; frumentum (froment, blé) ; gemma (pierre précieuse) ; germen (germe, semence) ; granum (grain) ; hedus (bouc, chevreau) ; hortus (jardin) ; laurea (feuille de laurier) ; lilium (lys) ; lignum (bois, bois de la croix) ; margarita (perle) ; oleum (huile) ; ovis (mouton, brebis) ; palma (palme) ; refloreo (refleurir) ; semino (semer) ; serpentinus (de serpent) ; sidereus (d’étoile, astral) ; sol (soleil) ; stella (étoile) ; vinea (la vigne) ; vinum (le vin) ; zizania (ivraie, zizanie (plante)).'
objets = 'aculeus (trait, projectile) ; arca federis (l’arche d’alliance) ; candelabrum (candélabre) ; cathedra (chaire, cathèdre, trône) ; clavis (clé) ; clipeus (bouclier) ; corona (couronne) ; crux (croix) ; diadema (diadème, couronne) ; documentum (document, charte) ; domus (maison, église) ; gubernaculum (gouvernail) ; habena (les rênes) ; jaculum (trait, javelot, lance) ; lapis (pierre) ; laqueus (filet, lac, rets) ; ligamen (lien) ; modius (boisseau) ; sceptrum (sceptre) ; sigillum (sceau) ; solium (trône) ; speculum (miroir) ; tabernaculum (tabernacle) ; talentum (talent (mesure monétaire)) ; torus (lit, couche) ; trames (trame, chemin) ; unguentum (onguent, baume) ; vinculum (lien) ; virga (verge, bâton pour frapper).'
espace = 'ager (champ) ; anfractus (crevasse, fossé) ; angustia (resserrement) ; apex (sommet, faîte) ; aqua (eau) ; arx (forteresse) ; aula (cour, cour royale) ; capella (chapelle) ; castrum (château) ; circuitus (circuit, quatre horizons) ; civitas (cité) ; claudo (enclore) ; claustrum (cloître) ; clima (les climats géographiques) ; conclusus (renfermé dans) ; conventus (couvent) ; cumulus (comble, faîte) ; curia (cour royale, céleste) ; curriculum (cercle, course) ; domanium (domaine) ; domus (maison, église) ; ecclesia (église) ; fastigium (faîte, comble) ; flumen (fleuve) ; fortalicium (fort) ; forum (marché) ; fretum (flots marins) ; fundamenta, orum (fondements) ; hortus (jardin) ; horreum (grenier) ; limen (seuil) ; limes (limite, frontière, borne) ; mons (montagne) ; mundus (le monde) ; munimen (rempart) ; oppidum (château, forteresse, place forte) ; orbis (le monde, l’orbe des terres) ; provincia (province) ; regio (région) ; robur (chêne, force) ; sanctuarium (sanctuaire) ; sedis (siège, demeure) ; semita (sentier, chemin) ; spaciosus (vaste) ; terra (terre) ; terrena (les choses terrestres) ; trames (trame, chemin) ; universus (l’univers) ; urbs (ville, ville capitale) ; vallis (vallée, val) ; vestigium (trace, vestige) ; via (voie, route) ; villa (bourg, petite ville)'
temps = 'abeo (s’effacer, devenir caduc) ; abolim (jadis, naguère) ; antecessor (prédécesseur, ancêtre) ; antiquatus (depuis les temps anciens) ; antiquitas, antiquus (des temps jadis, des temps reculés) ; assidue (régulièrement) ; assiduus (régulier) ; canus (chenu) ; celer (rapide, prompt) ; celeriter (rapidement, promptement) ; cito (id.) ; condo (fonder) ; conservo, conservatio (conserver, conservation) ; constans, constantia (constant, constance) ; consuesco (être habitué à, avoir coutume de, suivre la coutume de) ; consuetudo (coutume) ; continuatus (continué) ; continue (continûment) ; continuo (continuer) ; continuus (continu) ; cotidianus (quotidien) ; cotidie (quotidiennement) ; curriculum (cours de la vie) ; cursus (course du temps) ; deinde (ensuite) ; demum (enfin) ; dies (jour) ; diu (longtemps) ; diutius (très longtemps) ; diuturnus (long) ; etas (âge) ; eternaliter (éternellement) ; eternus (éternel) ; exordium (début) ; extremus (final, absolument dernier) ; frequenter (souvent, fréquemment) ; frequentius (très fréquemment) ; frequento (visiter souvent) ; futurum (futur) ; futurus (futur) ; gesta (les gestes du passé) ; gestus (id.) ; immortalitas (immortalité) ; imposterum (par la suite) ; incessanter (sans trêve) ; incipio (commencer) ; indeficienter (sans trêve) ; indesinenter (id.) ; inextinguibilis (qui ne peut être éteinte, arrêtée) ; invigilo (veiller à longueur de temps) ; jam (désormais, dans le passé) ; jugis (permanent) ; jugiter (en permanence) ; labens (fugace) ; labilitas (labilité, caractère fugace et destructeur du temps) ; maneo (rester, persister) ; manuteneo (maintenir) ; maturus (mûrement réfléchi) ; matutinus (matutin, matinal) ; memini (se souvenir) ; memor (qui se souvient) ; memoro (se souvenir) ; memoria (mémoire) ; memoriter (avec mémoire, par la mémoire) ; mora (arrêt, station, séjour) ; mortalis (mortel) ; mortalitas (mortalité) ; mortiferus (qui apporte la mort) ; natales (naissance) ; novissimus (tout dernier, arrivé en tout dernier lieu) ; novitas (nouveauté indue apportées par le temps) ; noviter (naguère, dernièrement) ; novus (neuf) ; nox (nuit) ; nuper (naguère) ; obitus, i (trépas) ; oblivio (oubli) ; obliviscor (oublier) ; origo (origine) ; orior (naître) ; permaneo (rester) ; perpetuo (perpétuellement) ; perpetuus (perpétuel) ; pervigilis (en veille perpétuelle) ; pervigilus (id.) ; postea (ensuite) ; posterus (suivant, qui succède) ; postmodum (dans un second temps) ; preambulus (qui marche en avant, qui précède, qui introduit) ; presens (du présent, actuel, présent) ; preteritus (du passé, passé) ; primevus (originel, du premier âge) ; primogenitura (primogéniture) ; primogenitus (premier né) ; primus (originel, premier) ; principium (principe, début, origine) ; prior (précédent) ; prioritas (la précédence et l’autorité des premiers) ; pristinus (ancien, et correct parce qu’ancien) ; prius (auparavant) ; promptior (plus rapide, très rapide) ; promptissimus (le plus rapide) ; promptius (très rapide) ; promptus (rapide) ; quondam (jadis, feu : feu les rois de France) ; refloreo (refleurir, après une période de décadence) ; rememoro (se remémorer) ; renovo (remettre à neuf, renouveler, rénover) ; resurrectio (résurrection) ; ruina (ruine, dégradation) ; semper (toujours) ; sempiternus (éternel) ; senectus (vieillesse) ; sepe (souvent) ; sepius (très souvent) ; soleo (faire couramment, avoir coutume de) ; successor (successeur) ; temporalia (biens, occupations, etc., temporels) ; temporaliter (dans le cours du temps, par opposition aux celestia intemporels) ; tempus (temps) ; terminus (terme fixe, terme définitif) ; ulterius (dans le futur, dorénavant) ; vestigia (traces laissées dans le passé) ; vestutas (vieillesse, délabrement) ; vetus (vieux).'

In [14]:
abondance = topic_list(abondance)
joie = topic_list(joie)
amour = topic_list(amour)
verticalité = topic_list(verticalité)
corps = topic_list(corps)
Dieu = topic_list(Dieu)
jugement = topic_list(jugement)
gouvernement = topic_list(gouvernement)
grâce = topic_list(grâce)
faute = topic_list(faute)
guerre = topic_list(guerre)
imitatio = topic_list(imitatio)
chaleur = topic_list(chaleur)
noblesse = topic_list(noblesse)
paix = topic_list(paix)
connaissance = topic_list(connaissance)
prière = topic_list(prière)
roi = topic_list(roi)
royaume = topic_list(royaume)
serviteur = topic_list(serviteur)
musique = topic_list(musique)
biodiversite = topic_list(biodiversite)
objets = topic_list(objets)
espace = topic_list(espace)
temps = topic_list(temps)

In [15]:
topic_dico = {'abondance':abondance,'joie':joie,'amour':amour,'verticalité':verticalité,'corps':corps,
             'dieu':Dieu,'jugement':jugement,'gouvernement':gouvernement,'grâce':grâce,'faute':faute,
             'guerre':guerre,'imitatio':imitatio,'chaleur':chaleur,'noblesse':noblesse,'paix':paix,
             'connaissance':connaissance,'prière':prière,'roi':roi,'royaume':royaume,'serviteur':serviteur,
             'musique':musique,'biodiversité':biodiversite,'objets':objets,'espace':espace,'temps':temps}
liste = list(topic_dico.values())
flat_list = [item for sublist in liste for item in sublist]

In [523]:
def find_key(word, dictionary):
    for key, value in dictionary.items():
        if word in value:
            return key
    return None

In [524]:
def find_keys(words, dictionary):
    keys = []
    for word in words:
        key = find_key(word, dictionary)
        if key is not None:
            keys.append(key)
        else:
            new_key = input(f"Enter a key for '{word}': ")
            if new_key not in dictionary:
                dictionary[new_key] = []
            dictionary[new_key].append(word)
            keys.append(new_key)
    return keys


In [527]:
keys = find_keys(result,topic_dico)
keys = [keys[i:i + len(keys) // 5] for i in range(0, len(keys), len(keys) // 5)]

In [8]:
df['Date'].to_list()

['1240',
 '1246',
 '1245',
 '1247',
 '1243',
 '1239',
 '1239',
 '1241',
 '1241',
 '1245',
 '1241',
 '1241',
 '1241',
 '1242',
 '1250',
 '1248',
 '1243',
 '1249',
 '1249',
 '1245',
 '1239',
 '1240',
 '1239',
 '1264',
 '1238-1239',
 '1231',
 '1231',
 '1244-1245',
 '1241',
 '1241',
 '1239',
 '1244-1245',
 '1243',
 '1237',
 '1243',
 '1238',
 '1241',
 '1248',
 '1241',
 '1241-1253',
 '1242',
 '1250',
 '1246',
 '1248',
 '1246',
 '1246',
 '1240',
 '1250',
 '1231',
 '1252',
 '1239',
 '1239',
 '1246',
 '1249',
 '1249',
 '1249',
 '1250',
 '1249',
 '1250',
 '1253',
 '1220-1266',
 '1253',
 '1253',
 '1253',
 '1253',
 '1250',
 '1249',
 '1237',
 '1240',
 '1247',
 '1241',
 '1238',
 '1248',
 '1248',
 '1248',
 '1252',
 '1248',
 '1254',
 '1254',
 '1247',
 '1248',
 '1247',
 '1237',
 '1248',
 '1246',
 '1243',
 '1243',
 '1243',
 '1243',
 '1247',
 '1243',
 '1247',
 '1236',
 '1249',
 '1236-1237',
 '1236-1237',
 '1236',
 '1220-1266',
 '1243',
 '1241',
 '1246',
 '1253',
 '1224',
 '1252',
 '1252-1253',
 '1248',
 

In [16]:
flat_list

['abundans',
 'abundanter',
 'abundantia',
 'abundantius',
 'abundo',
 'accresco',
 'adaugeo',
 'affluencia',
 'affluenter',
 'ager',
 'alimentum',
 'amplio',
 'amplior',
 'amplius',
 'augeo',
 'augmentatio',
 'augmento',
 'augmentum',
 'copia',
 'copiosior',
 'copiosius',
 'cresco',
 'diminuo',
 'diminutus',
 'dives',
 'egestas',
 'fertilitas',
 'fructuosus',
 'fructus',
 'frumentum',
 'germen',
 'germino',
 'granatus',
 'horreum',
 'hortus',
 'incrementum',
 'inedia',
 'liberalis',
 'liberalitas',
 'liberaliter',
 'liberalius',
 'locuples',
 'locupleto',
 'magnifice',
 'magnificentia',
 'magnifico',
 'magnificus',
 'magnitudo',
 'major',
 'meto',
 'munifice',
 'munificentia',
 'munificentior',
 'munificus',
 'oleum',
 'pauper',
 'paupertas',
 'pinguior',
 'prodigalitas',
 'profectus',
 'prospere',
 'prosperitas',
 'prosperius',
 'prospero',
 'prosperus',
 'ros',
 'semino',
 'sustento',
 'uber',
 'uberior',
 'uberius',
 'ubertas',
 'victualia',
 'victus',
 'vinum',
 'beatissimus',
 'b